In [8]:
import qualified Data.ByteString       as B
import qualified Data.ByteString.Char8 as BC
import Data.Maybe

transform (r:rs) = let
    edges = map (map (fst . fromJust . BC.readInt) . (r:) . BC.split ',') $ tail rs
    flipped = map (\[v,u,w] -> [u,v,w]) edges
    in edges ++ flipped

content <- concatMap (transform . BC.words) . BC.lines <$> B.readFile "dijkstraData.txt"

In [13]:
import Control.Monad.ST
import Control.Monad
import Data.STRef
import qualified Data.Vector.Unboxed.Mutable as M
import qualified Data.Vector.Unboxed as V

bellmanFord graph source = runST $ do
    let len = head (tail (last graph)) + 1
    dist <- M.replicate len 1000000
    M.write dist source 0
    forM_ [1..(len - 1)] $ \_ ->
        forM_ graph $ \[v, u, w] -> do
            distU <- M.read dist u
            distV <- M.read dist v
            when ((distU + w) < distV) $
                M.write dist v (distU + w)
    frozen <- V.freeze dist
    result <- newSTRef (Right frozen)
    forM_ graph $ \[v, u, w] -> do
        distU <- M.read dist u
        distV <- M.read dist v
        when (distU + w < distV) $
                writeSTRef result (Left "cycle found")
    readSTRef result

In [14]:
Right result = bellmanFord content 1

In [15]:
map (result V.!) [7,37,59,82,99,115,133,165,188,197]

[2599,2610,2947,2052,2367,2399,2029,2442,2505,3068]